# **Descrição**

*   **Repositório contendo a abordagem completa:** https://github.com/imor-de/microsoft_malware_prediction_kaggle_2nd

*   **Abordagem de codificação de dados utilizada:** https://github.com/imor-de/microsoft_malware_prediction_kaggle_2nd/blob/master/code/3_Data_Encoding_M1.ipynb

***
Durante esta etapa, MICHAELS; IMORDE argumenta que nem todos os métodos de aprendizado de máquina possuem a capacidade de lidar com qualquer tipo de característica (explicitamente características categóricas). E para isso, essas são codificadas para assegurar processamento adequado.

Para as características selecionadas, são utilizadas as abordagens de codificação por rótulos (Label Encoding) e codificação por frequências (Frequency Encoding). A codificação por rótulos transforma valores em números entre 0 e n-1 (sendo n o número de rótulos diferentes). Na codificação por frequência, os valores são codificados com base em suas frequências, sendo transformados em números entre 0 e m (onde m é o número de valores com frequência >= 2).
***

# **Bibliotecas/Funções**

In [ ]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from sklearn import preprocessing
from tqdm import tqdm
import gc
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [ ]:
# Especificação dos tipos de dados para menor uso de memória
dtypes = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float16',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int8',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float32',
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float16',
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float16',
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float32',
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float32',
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float16',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float16',
        'Census_InternalPrimaryDisplayResolutionVertical':      'float16',
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float32',
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }

In [ ]:
def convert_types(df):
    # Conversão dos tipos de dados para reduzir memória utilizada
    for c in df:
        col_type = str(df[c].dtypes)
        numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
        
        # Conversão de "object" para "category"
        if col_type == 'object':
            df[c] = df[c].astype('category')
        
        # Conversão de numéricos
        elif col_type in numerics:
            c_min = df[c].min()
            c_max = df[c].max()
            if col_type[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[c] = df[c].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[c] = df[c].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[c] = df[c].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[c] = df[c].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[c] = df[c].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[c] = df[c].astype(np.float32)
                else:
                    df[c] = df[c].astype(np.float64)  
        
    return df

# **Importação de Dados**

In [ ]:
# Definição da coluna alvo
target = 'HasDetections'

# Definição da coluna identificadora
data_id = 'MachineIdentifier'

In [ ]:
# Carregamento dos dados utilizando a biblioteca Dask para melhor performance
file = 'MMP_Cleaned.csv'
ddf = dd.read_csv(file, dtype = dtypes)
df = ddf.compute()

In [ ]:
# Forma do conjunto de dados (linhas, colunas)
df.shape

(8921483, 71)

In [ ]:
# Conversão dos tipos de dados para o conjunto remodelado
df = convert_types(df)

# **Codificação de Dados**

In [ ]:
# Build up a list with all the features, which should be encoded via frequency
list_frequency_encoding = ['AppVersion',
 'AvSigVersion',
 'Census_OSVersion',
 'EngineVersion',
 'OsBuildLab']

In [ ]:
# Construção da lista com todas as características que devem ser codificadas por rótulo
correct_feature_by_hand = ['AppVersion',
 'AvSigVersion',
 'Census_ActivationChannel',
 'Census_ChassisTypeName',
 'Census_DeviceFamily',
 'Census_FlightRing',
 'Census_GenuineStateName',
 'Census_MDC2FormFactor',
 'Census_OSArchitecture',
 'Census_OSBranch',
 'Census_OSEdition',
 'Census_OSInstallTypeName',
 'Census_OSSkuName',
 'Census_OSVersion',
 'Census_OSWUAutoUpdateOptionsName',
 'Census_PowerPlatformRoleName',
 'Census_PrimaryDiskTypeName',
 'EngineVersion',
 'MachineIdentifier',
 'OsBuildLab',
 'OsPlatformSubRelease',
 'OsVer',
 'Platform',
 'Processor',
 'SkuEdition',
 'SmartScreen',
 'Census_MDC2FormFactor_new']

list_label_encoding = list(set(correct_feature_by_hand)-set(list_frequency_encoding))

In [ ]:
# Remoção da coluna alvo da lista "list_label_encoding"
list_label_encoding.remove(data_id)

In [ ]:
# Create a function for frequency encoding
def frequency_encoding(feature):
    # Count the number of values of each feature and reset the indices
    t = df[feature].value_counts().reset_index()
    # Building up a new index (old index is set by default to 'level_0')
    t = t.reset_index()
    # Set the old index 'level_0' for all values, which only occur once, to NaN
    t.loc[t[feature] == 1, 'level_0'] = np.nan
    # Reset the original index (= the value name) as index
    t.set_index('index', inplace=True)
    # return the number of values , which occur two or more times, +1
    max_label = t['level_0'].max() + 1
    # fill all nan-values to max_label
    t.fillna(max_label, inplace=True)
    return t.to_dict()['level_0']

In [ ]:
# Codificação de todas as características contidas na lista "list_frequency_encoding"
for feature in tqdm(list_frequency_encoding):
    freq_enc_dict = frequency_encoding(feature)
    df[feature] = df[feature].map(lambda x: freq_enc_dict.get(x, np.nan))
    df[feature] = df[feature].astype('int64')    

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  2.70it/s]


In [ ]:
# Codificação de todas as características contidas na lista "list_label_encoding"
for feature in tqdm(list_label_encoding):
    le = preprocessing.LabelEncoder()
    df[feature] = le.fit_transform(df[feature])
    df[feature] = df[feature].astype('int64') 

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:37<00:00,  1.79s/it]


# **Exportação de Dados**

In [ ]:
# Exportação do conjunto de dados para o arquivo "train_cleaned_encoded.csv"
df.to_csv('MMP_Cleaned_Encoded.csv', index = False)